<a href="https://colab.research.google.com/github/ArindamBanerji/wip-experiments/blob/master/misc/run_streamlit_TS_Image_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# set some variables
base_dir = "/content/gdrive/My Drive/python-projects/kaggle_experiments"
repo_git = "https://github.com/ArindamBanerji/Time-Series.git"
local_repo = "./Time-Series"

# set forecast type here
forecast_type = "finance"

# based on forecast type set the variable values
if forecast_type == "weather":
  source_base = "./Time-Series/weather_data"
  fnm = "./streamlit-weather-forecasting-app.py"
else:
  source_base = "./Time-Series/finance_data_v2"
  fnm = "./financial-forecasting-streamlit-app.py"

In [3]:
# go to the correct directory
import os
os.chdir(base_dir)

In [4]:
!ls

streamlit_code	Time-Series


In [5]:
! (test -d $local_repo && git -C $local_repo pull --rebase) || git clone $repo_git

Already up to date.


In [10]:
# Approach - 2 - Idempotent pulls
! DIR=./Time-Series && ORIGIN="https://github.com/ArindamBanerji/Time-Series.git" && \
  (test -d $DIR && git -C $DIR pull --rebase) || git clone $ORIGIN $DIR

Already up to date.


In [6]:
! pip install streamlit -q

In [7]:
! pip install pyngrok

In [8]:
from pyngrok import ngrok

In [9]:
# extract auth-token from secrets
from google.colab import userdata
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
# ngrok_authtoken

In [10]:
# Add your ngrok token here
ngrok.set_auth_token(ngrok_authtoken)

In [11]:
os.chdir(source_base)

! pip install -r requirements.txt

In [12]:
! (test -f $fnm && echo "file-exists") || echo "file-not-exists"

file-exists


In [13]:
!nohup streamlit run "{fnm}" --server.port 8501 &
# Configure the tunnel with the correct format
tunnel = ngrok.connect(8501, "http")
print(tunnel.public_url)

nohup: appending output to 'nohup.out'
https://ff1c-34-16-189-192.ngrok-free.app


In [14]:
ngrok.kill()

Slightly more structured way to do the same thing as above - using threads in stead of **nohup**

In [15]:
streamlit_cmd = f'streamlit run {fnm} --server.port 8501'
streamlit_cmd

'streamlit run ./financial-forecasting-streamlit-app.py --server.port 8501'

In [16]:
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system(streamlit_cmd)

In [17]:
# Start a thread to run the Streamlit app
from threading import Thread
thread = Thread(target=run_streamlit)
thread.start()

In [18]:
# Open a tunnel to the streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://992d-34-16-189-192.ngrok-free.app" -> "http://localhost:8501"


In [19]:
ngrok.kill()